In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import nltk
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import nltk
import unicodedata
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import lightgbm as lgb
import csv
import re
from collections import Counter

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Load the data
df_train = pd.read_csv('/content/drive/MyDrive/Data/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Data/test.csv')

In [ ]:
df_train.head(5)

,Label,Content
0,2,second counting input 5 2 which receives inter...
1,4,extremely low temperature of the chips in cold...
2,3,of the basic ammonium salt of the carboxyl ate...
3,9,18 u2033 is provided which is axially supporte...
4,2,to an u201c inner surface u201d means the surf...


In [ ]:
df_train['Content'] = df_train['Content'].str.lower()
df_test['Content'] = df_test['Content'].str.lower()

# **Remove Punctuation

In [ ]:
import string
def remove_punctuations(text):
  punctuation = string.punctuation
  return text.translate(str.maketrans("","",punctuation))

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
df_train['Content'] = df_train['Content'].apply(lambda x: remove_punctuations(x))
df_test['Content'] = df_test['Content'].apply(lambda x: remove_punctuations(x))

Remove of stopwords

In [ ]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [ ]:
stopwords = set(stopwords.words('english'))
def remove_stopwords(text):
  return " ".join([word for word in text.split() if word not in stopwords])

In [ ]:
df_train['Content'] = df_train['Content'].apply(lambda x: remove_stopwords(x))
df_test['Content'] = df_test['Content'].apply(lambda x: remove_stopwords(x))

In [ ]:
df_train['Content']

0        second counting input 5 2 receives internally ...
1        extremely low temperature chips cold seasons c...
2        basic ammonium salt carboxyl ate water soluble...
3        18 u2033 provided axially supported crankshaft...
4        u201c inner surface u201d means surface layer ...
                               ...                        
59114    skilled art appreciate various filling securin...
59115    work data typically encrypted encryption algor...
59116    supply side reel table 2 spring 6 brake arm 4 ...
59117    detent roller 84 detent element 80 engagement ...
59118    laptop computer rack mount computer system des...
Name: Content, Length: 59119, dtype: object

# remove most frequent words

In [ ]:
from collections import Counter

word_count = Counter()
for text in df_train['Content']:
    for word in text.split():
        word_count[word] += 1

word_count.most_common(10)

[('1', 62692),
 ('may', 62402),
 ('2', 58983),
 ('shown', 41259),
 ('10', 40133),
 ('3', 39601),
 ('one', 38617),
 ('invention', 34429),
 ('5', 33139),
 ('4', 32961)]

In [ ]:
from collections import Counter

word_count1 = Counter()
for text in df_test['Content']:
    for word in text.split():
        word_count1[word] += 1

word_count1.most_common(10)

[('1', 27264),
 ('may', 26347),
 ('2', 25798),
 ('shown', 18168),
 ('10', 17846),
 ('3', 17374),
 ('one', 16209),
 ('5', 14723),
 ('invention', 14452),
 ('4', 14363)]

In [ ]:
# remove most frequent words
frequent_words = set(word for (word,wc) in word_count.most_common(10))
def remove_freq_words(text):
  return " ".join([word for word in text.split() if word not in frequent_words])

# remove most frequent words
frequent_words1 = set(word for (word,wc) in word_count1.most_common(10))
def remove_freq_words1(text):
  return " ".join([word for word in text.split() if word not in frequent_words1])


In [ ]:
df_train['Content'] = df_train['Content'].apply(lambda x: remove_freq_words(x))
df_test['Content'] = df_test['Content'].apply(lambda x: remove_freq_words1(x))

# removal of of rare words

In [ ]:
rare_words_train = set((word,wc) for (word,wc) in word_count.most_common()[:-10:-1])
rare_words_train

{('201414314296', 1),
 ('airstroke', 1),
 ('congestire', 1),
 ('endopeptidase', 1),
 ('hyperaldosteronemia', 1),
 ('interband', 1),
 ('natriuresis', 1),
 ('photoprocessing', 1),
 ('toolbars', 1)}

In [ ]:
rare_words_test = set((word,wc) for (word,wc) in word_count1.most_common()[:-10:-1])
rare_words_test

{('adrienne', 1),
 ('boroditsky', 1),
 ('cedric', 1),
 ('feuer', 1),
 ('misha', 1),
 ('oxiom', 1),
 ('popularization', 1),
 ('sheryl', 1),
 ('wbb2', 1)}

In [ ]:
rare_words_train = set(word for (word,wc) in word_count.most_common()[:-10:-1])
rare_words_train

{'201414314296',
 'airstroke',
 'congestire',
 'endopeptidase',
 'hyperaldosteronemia',
 'interband',
 'natriuresis',
 'photoprocessing',
 'toolbars'}

In [ ]:
rare_words_test = set(word for (word,wc) in word_count1.most_common()[:-10:-1])
rare_words_test

{'adrienne',
 'boroditsky',
 'cedric',
 'feuer',
 'misha',
 'oxiom',
 'popularization',
 'sheryl',
 'wbb2'}

In [ ]:
def remove_rare_words_train(text):
  return " ".join([word for word in text.split() if word not in rare_words_train])

In [ ]:
df_train['Content'] = df_train['Content'].apply(lambda x: remove_rare_words_train(x))


In [ ]:
def remove_rare_words_test(text):
  return " ".join([word for word in text.split() if word not in rare_words_test])

In [ ]:
df_test['Content'] = df_test['Content'].apply(lambda x: remove_rare_words_test(x))

# Remove Special Character

In [ ]:
import re
def remove_spl_char(text):
  text = re.sub('[^a-zA-Z0-9]',' ',text)
  text = re.sub('\s+',' ',text)
  return text

In [ ]:
df_train['Content'] = df_train['Content'].apply(lambda x: remove_spl_char(x))
df_test['Content'] = df_test['Content'].apply(lambda x: remove_spl_char(x))

In [ ]:
df_train['Content']

0        second counting input receives internally cont...
1        extremely low temperature chips cold seasons c...
2        basic ammonium salt carboxyl ate water soluble...
3        18 u2033 provided axially supported crankshaft...
4        u201c inner surface u201d means surface layer ...
                               ...                        
59114    skilled art appreciate various filling securin...
59115    work data typically encrypted encryption algor...
59116    supply side reel table spring 6 brake arm brak...
59117    detent roller 84 detent element 80 engagement ...
59118    laptop computer rack mount computer system des...
Name: Content, Length: 59119, dtype: object

# Stemming

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem_words(text):
  return " ".join([ps.stem(word) for word in text.split()])


In [ ]:
df_train['Content'] = df_train['Content'].apply(lambda x: stem_words(x))
df_test['Content'] = df_test['Content'].apply(lambda x: stem_words(x))

# lemmatization & pos tagging

In [ ]:
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
print(wordnet.NOUN)
print(wordnet.VERB)

n
v


In [ ]:
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N" : wordnet.NOUN,"V": wordnet.VERB, "J": wordnet.ADJ, "R": wordnet.ADV}
def lemmatize_words(text):
  #find pos tag
  pos_text  = pos_tag(text.split())
  return " ".join([lemmatizer.lemmatize(word,wordnet_map.get(pos[0],wordnet.NOUN)) for word,pos in pos_text])

In [ ]:
df_train['Content'] = df_train['Content'].apply(lambda x: lemmatize_words(x))
df_test['Content'] = df_test['Content'].apply(lambda x: lemmatize_words(x))

# unicodedata

In [ ]:
def remove_unicode_chars(text):

    # Define the Unicode characters to remove (you can adjust this list)
    chars_to_remove = ['u00b0', "u2033", "u201c", "u201d", "u2014", "u03c1", "u03bc", "u03b8","u00e5","a0f","a2f","u2192","a7r","u03b3","u2212","u2032","u03b4"]

    # Replace each Unicode character with an empty string
    for char in chars_to_remove:
        text = text.replace(char, "")

    return text

In [ ]:
df_train['Content'] = df_train['Content'].apply(lambda x: remove_unicode_chars(x))
df_test['Content'] = df_test['Content'].apply(lambda x: remove_unicode_chars(x))

In [ ]:
df_test.head(25)

,Label,Content
0,NaN,tube 14 deflector 22 form cylindr insid wall s...
1,NaN,surfac 18 contact ball 14 pressur act 24 press...
2,NaN,fig7 show right tubular needl 75 releas natur ...
3,NaN,rel small annular surfac forward end collar 42...
4,NaN,interfac substrat quartz glass film adhes poro...
5,NaN,whether sheet move give distanc six meter dete...
6,NaN,glass object result product layer form provid ...
7,NaN,respect arrang directli cutter holder 112 plur...
8,NaN,cut blade receiv cassett tape cut blade cut ta...
9,NaN,opposit spheric protrus 51 rotari movement say...


# **correct_spelling**

In [ ]:
pip install pyspellchecker

In [ ]:
# Importing necessary libraries
from spellchecker import SpellChecker

# Defining the correct_spelling function
def correct_spelling(text):

    # Initialize the spell checker
    spell = SpellChecker()

    # Split the text into words
    words = text.split()

    # Correct each word
    corrected_words = []
    for word in words:
        # Get the corrected version of the word, or the original word if not found
        corrected_word = spell.correction(word) or word
        corrected_words.append(corrected_word)

    # Join corrected words back into text
    corrected_text = " ".join(corrected_words)

    return corrected_text

In [ ]:
df_train['Content'] = df_train['Content'].apply(lambda x: correct_spelling(x))

In [ ]:
from tqdm import tqdm

# Define your DataFrame df_train and the correct_spelling function here

# Get the total number of rows in the DataFrame
total_rows = len(df_test)

# Create a progress bar with tqdm
with tqdm(total=total_rows) as pbar:
    # Define a function to apply to each row of the DataFrame
    def apply_correction(row):
        # Your correct_spelling function
        corrected_text = correct_spelling(row['Content'])
        # Update the 'Content' column with the corrected text
        row['Content'] = corrected_text
        # Update the progress bar
        pbar.update(1)
        # Return the modified row
        return row

    # Apply the apply_correction function to each row of the DataFrame
    df_train = df_train.apply(apply_correction, axis=1)

# After the loop is done, df_train will contain the corrected 'Content' column


In [ ]:
df_test['Content'] = df_test['Content'].apply(lambda x: correct_spelling(x))

In [ ]:
import pandas as pd

# Define the file name for the Excel file
excel_file_name = "/content/drive/MyDrive/Data/df_train_clean.xlsx"

# Save the DataFrame to an Excel file
df_train.to_excel(excel_file_name, index=False)

print(f"Excel file '{excel_file_name}' has been created.")


In [ ]:
import pandas as pd

# Define the file name for the Excel file
excel_file_name = "/content/drive/MyDrive/Data/df_train_clean.xlsx"

# Save the DataFrame to an Excel file
df_test.to_excel(excel_file_name, index=False)

print(f"Excel file '{excel_file_name}' has been created.")


In [ ]:
# Load the data
df_train_2 = pd.read_csv("/content/drive/MyDrive/Split_data/train_data_2.csv")

In [ ]:
len(df_train_2)

29560

# **fasttext** : Spell Correct Fast

In [ ]:
!pip install fasttext

In [ ]:
import fasttext

# Train FastText model
model = fasttext.train_unsupervised("/content/drive/MyDrive/Split_data/train_data_2.csv", model='skipgram')

# Save the trained model
model.save_model('/content/drive/MyDrive/Data/model_Train_2.bin')

# Now you can use model.load_model('model.bin') to load the model

In [ ]:
!pip install swifter


In [ ]:
import swifter
from tqdm.auto import tqdm

# Get the total number of rows in the DataFrame
total_rows = len(df_train_2)

# Define a function to correct spelling using FastText
def correct_spelling_fasttext(text):
    corrected_text = []
    for word in text.split():
        corrected_word = model.get_nearest_neighbors(word, k=1)[0][1]
        corrected_text.append(corrected_word)
    return ' '.join(corrected_text)

# Apply the correct_spelling_fasttext function to each element of the 'Content' column in parallel with swifter
with tqdm(total=total_rows) as pbar:
    def apply_correction_fasttext(text):
        pbar.update(1)  # Update the progress bar for each row processed
        return correct_spelling_fasttext(text)

    df_train_2['Content'] = df_train_2['Content'].swifter.apply(apply_correction_fasttext)

# After this operation, df_train['Content'] will contain the corrected text


In [ ]:
df_test.head(5)

,Label,Content
0,NaN,",tube deflectors ,formed ,cylindrical ,inside ..."
1,NaN,",surface ,contact ,ball ,pressure detracting ,..."
2,NaN,"fig6 ,shows left ,tubular ,needle releases ,na..."
3,NaN,",relatively ,small ,annular ,surface ,forward ..."
4,NaN,",interface ,substrate yag ,glass ,film nonadhe..."


In [ ]:
def remove_commas(text):

    return text.replace(',', '')

# Example usage:
text_with_commas = "This, is, a, text, with, commas."
text_without_commas = remove_commas(text_with_commas)
print(text_without_commas)


This is a text with commas.


In [ ]:
df_test['Content'] = df_test['Content'].apply(lambda x: remove_commas(x))

In [ ]:
df_test.head(5)

,Label,Content
0,NaN,tube deflectors formed cylindrical inside wall...
1,NaN,surface contact ball pressure detracting press...
2,NaN,fig6 shows left tubular needle releases natura...
3,NaN,relatively small annular surface forward endwa...
4,NaN,interface substrate yag glass film nonadhesive...


In [ ]:
import pandas as pd

# Define the file name for the CSV file
csv_file_name = "/content/drive/MyDrive/clean/df_test_clean.csv"

# Save the DataFrame to a CSV file
df_test.to_csv(csv_file_name, index=False)

print(f"CSV file '{csv_file_name}' has been created.")

CSV file '/content/drive/MyDrive/clean/df_test_clean.csv' has been created.
